In [1]:
import requests

def request_rasa(query):
    try:
    
        payload = {"sender": "Rasa", "text": query}
        headers = {'content-type': 'application/json'}
        response = requests.post('http://localhost:5005/model/parse', json=payload, headers=headers)
        response = response.json()
        print("response :\n", response)
    except Exception as e:
        print("An error occurred:", str(e))

    return response


def fetch_rasa_json(response):
    intent=response['intent']['name']
    entity=response['entities'][0]['value']
    return {'intent':intent,'entite':entity} 
intents_dont_require_graph=['ask_definition','ask_synonym','ask_keyword','ask_figure']

def fetch_query(rasa_query):
    intent=rasa_query['intent']
    entite=rasa_query['entite']
    if intent in intents_dont_require_graph:
        pipeline_type=1   # pipeline_type represent not needing knowledge graph 
    else:
        pipeline_type=2   # pipeline_type require navigating the knowledge graph
    return intent,entite,pipeline_type    
        

In [2]:
fetch_query({'intent': 'ask_definition', 'entite': 'risk'})

('ask_definition', 'risk', 1)

In [3]:
def verfier_concept_exit(entite,knowledge_base): # check if entite exist in 
    return entite in knowledge_base["name"].values
def get_specifique_info(entite,intent,knowledge_base):
    mask = knowledge_base["name"] == entite
    if mask.any():
        row= knowledge_base[mask].iloc[0] 
        if 'definition' in intent:
            return row['description']
        elif 'keyword' in intent:
            return row['rake_keyword']
        elif 'synonym' in intent:
            return row['synonym']
        elif 'figure' in intent:
            return row['Figure']
            

    else:
        return None
    
def get_most_similar_concept(entity, Knowledge_base):
    concept_list=knowledge_base['name'].tolist()

 
    max_similarity = -1 
    most_similar_concept = None

    for concept in concept_list:
        try:
            similarity = word2vec_model.wv.similarity(entity, concept)
            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_concept = concept
        except KeyError:
            # Handle the case where either entity or concept is not present in the vocabulary
            continue

    return most_similar_concept


def get_all_nodes_in_relationship_with(entite,edges_liste):    
    related_relationships = [edge for edge in edges_liste if edge['head'] == entite]
    return related_relationships
import pandas as pd
def get_detailed_info(nodes_liste, knowledge_base,filter_var=True):
   
    detailed_info_list = []

    for node in nodes_liste:
        head_info = knowledge_base[knowledge_base['name'] == node['head']]
        tail_info = knowledge_base[knowledge_base['name'] == node['tail']]

        if not head_info.empty and not tail_info.empty:
            if filter_var==True:
                if head_info['importance'].iloc[0]>=tail_info['importance'].iloc[0]:
                    detailed_info_list.append({'head':head_info.iloc[0],'type':node['type'],'tail':tail_info.iloc[0]})
            else:
                detailed_info_list.append({'head':head_info.iloc[0],'type':node['type'],'tail':tail_info.iloc[0]})

                
        

    return detailed_info_list
    
    

In [4]:
def pipeline_manager(pipeline_type,intent,entite):
    if pipeline_type==1: # section represent calling one concept with only one feature
        concept_existince=verfier_concept_exit(entite,knowledge_base) # verfie if a the entites already exist in the knowledge base
        if concept_existince : # if the entite exist in the concept list
            return get_specifique_info(entite,intent,knowledge_base)
        
        if not concept_existince:
            
            simiar_entite=get_most_similar_concept(entite,knowledge_base) # use the synonym dictionary (word2vec) to get the most similar concept to the entite(input)
            print(simiar_entite)

            return get_specifique_info(simiar_entite,intent,knowledge_base)
    if pipeline_type==2:
        concept_existince=verfier_concept_exit(entite,knowledge_base) 
        
        if concept_existince : # if the entite exist in the concept list
            nodes_liste=get_all_nodes_in_relationship_with(entite,edges_liste) # get liste of nodes in relationship with the main entite based on certain depth
            nodes_info=get_detailed_info(nodes_liste,knowledge_base) # get the features of each node from the knowledge base and use filter_var , to filter for importance to maintain the structure
            return nodes_info
            
            
         
        if not concept_existince:
            simiar_entite=get_most_similar_concept(entite,knowledge_base) 
            print(simiar_entite)
            nodes_liste=get_all_nodes_in_relationship_with(simiar_entite,edges_liste)
            nodes_info=get_detailed_info(nodes_liste,knowledge_base)
            return nodes_info
        
def build_concepts_from_node(nodes):
    relatedconcept=[]
    if len(nodes)>0:
        mainconcept=serie_to_dic(nodes[0]['head'])
    for node in nodes:
        dic=serie_to_dic(node['tail'])
        dic['relationship']=node['type']
        relatedconcept.append(dic)
    if len(nodes)==0:
        mainconcept={}   
        relatedconcept={}
    return mainconcept,relatedconcept 
def serie_to_dic(serie):
    dic={}
    dic['name']=serie['name']
    dic['figure']=serie['Figure']
    dic['description']=serie['description']
    dic['key_word']=serie['rake_keyword']
    dic['synonym']=serie['synonym']
    return dic            
            
from gensim.models import Word2Vec
import pandas as pd
synonym = Word2Vec.load("synoynm_embedding.bin")
word2vec_model=synonym        
# declate knowledge_base , entites_relationships
knowledge_base=pd.read_csv('results_6.csv')
knowledge_base=knowledge_base.drop(['Content','parent_child_relationship','relationship_entites_final_1','definition','reference',
       ],axis=1)
import pickle

# Load the list from the file using pickle
with open('edge_liste.pkl', 'rb') as file:
    edges_liste = pickle.load(file)


In [5]:
def chatbot_reponse(query,simple=True):
    query=query.lower()
    reponse=request_rasa(query)
    fetched_request=fetch_rasa_json(reponse)
    intent,entite,pipeline_type=fetch_query(fetched_request)
    if entite =='plan risk':
        entite='plan risk management'
    if simple==True and pipeline_type==2:
        return 'type_1'
    if simple==False and pipeline_type==1:
        return 'type_2'
    output=pipeline_manager(pipeline_type,intent,entite)
    return output,intent
    
    

In [6]:
# test pipeline
    # test existing entites
        # definiton
        # synonym
        # key_word
        # figures
        # explain detailed process
    # test simiarity 
        # same process

In [7]:
knowledge_base = knowledge_base.drop_duplicates(subset='name', keep='first')
import ast
knowledge_base['Figure'] = knowledge_base['Figure'].fillna('')

knowledge_base['Figure'] = knowledge_base['Figure'].apply(lambda x: ast.literal_eval(x) if x else [])

In [8]:
display(knowledge_base)


,Concept,Intro,description,Figure,Table,name,rake_keyword,synonym,importance
0,11.1 PLAN RISK MANAGEMENT,11.1 PLAN RISK MANAGEMENT\nPlan Risk Managemen...,plan risk management plan risk management is t...,"{Figure 11-3, Figure 11-2}",set(),plan risk management,"['conduct risk management activities', 'risk m...","['risk management process', 'inﬂuence plan ris...",0.8
1,11.2 IDENTIFY RISKS,11.2 IDENTIFY RISKS\nIdentify Risks is the pro...,identify risks identify risks is the process o...,"{Figure 11-7, Figure 11-6}",set(),identify risks,"['perform qualitative risk analysis process', ...","['risk also', 'risks process', 'matter experts...",0.8
2,11.3 PERFORM QUALITATIVE RISK ANALYSIS,11.3 PERFORM QUALITATIVE RISK ANALYSIS\nPerfor...,perform qualitative risk analysis perform qual...,"{Figure 11-9, Figure 11-8}",set(),perform qualitative risk analysis,"['perform qualitative risk analysis process',...","['qualitative risk analysis process', 'identiﬁ...",0.8
3,11.4 PERFORM QUANTITATIVE RISK ANALYSIS,11.4 PERFORM QUANTITATIVE RISK ANALYSIS\nPerfo...,perform quantitative risk analysis perform qua...,"{Figure 11-12, Figure 11-11}",set(),perform quantitative risk analysis,['provide additional quantitative risk informa...,"['project documents project management', 'docu...",0.8
4,11.5 PLAN RISK RESPONSES,11.5 PLAN RISK RESPONSES\nPlan Risk Responses ...,plan risk responses plan risk responses is the...,"{Figure 11-17, Figure 11-16}",set(),plan risk responses,"['structured decision-making techniques', 'tre...","['risk responses process', 'assets project man...",0.8
...,...,...,...,...,...,...,...,...,...
293,schedule management plan,NaN,schedule management plan. described in section...,[],NaN,schedule management plan,['schedule management plan'],"['cost management plan', 'quality management p...",0.2
300,lessons learned repository,NaN,NaN,[],NaN,lessons learned repository,['lessons learned repository'],NaN,0.2
305,fallback plan,NaN,NaN,[],NaN,fallback plan,['fallback plan'],NaN,0.2
306,risk responses,NaN,NaN,[],NaN,risk responses,['risk responses'],"['agreed upon', 'agreedupon risk', 'process ri...",0.2


In [9]:
list(knowledge_base.iloc[2]['Figure'])[0]

'Figure 11-9'

In [10]:
nodes=pipeline_manager(1,'ask_keyword','risk responses'.lower())

In [11]:
nodes

"['risk responses']"

In [12]:
#dic_concepts=build_concepts_from_node(nodes)
#dic_concepts[1]

In [13]:
dic_figure={ 
'Figure 11-1':'https://ibb.co/1MYB82R',
'Figure 11-2':"https://ibb.co/TYHQ2mZ",
'Figure 11-3':'https://ibb.co/W2x0jLh',
'Figure 11-4':'https://ibb.co/cCYStgf',
'Figure 11-5':'https://ibb.co/whLGf8R',
'Figure 11-6':'https://ibb.co/QndtT3m',
'Figure 11-7':"https://ibb.co/RyFfrWd",
'Figure 11-8':"https://ibb.co/M2XqDMx",
'Figure 11-9':"https://ibb.co/gzr1CpM",
'Figure 11-10':"https://ibb.co/GVL4fMJ",
'Figure 11-11':"https://ibb.co/rQZpvNF",
'Figure 11-12':"https://ibb.co/3RJtzWv",
'Figure 11-13':"https://ibb.co/N7T45MS",
'Figure 11-14':"https://ibb.co/xS7BL95",
'Figure 11-15':"https://ibb.co/2k7tSBd",
'Figure 11-16':"https://ibb.co/LrCgpwR",
'Figure 11-17':"https://ibb.co/NNxKQfR",
'Figure 11-18':"https://ibb.co/smD216N",
'Figure 11-19':"https://ibb.co/MCdccKr",
'Figure 11-20':"https://ibb.co/TPCsZNn",
'Figure 11-21':"https://ibb.co/jRL74Qn"
}

In [14]:
dicss={'Figure 11-18', 'Figure 11-19'}
figure_output={}
for i in list(dicss):
    figure_output[i]=dic_figure[i]
figure_output    

{'Figure 11-19': 'https://ibb.co/MCdccKr',
 'Figure 11-18': 'https://ibb.co/smD216N'}

In [15]:
from flask import Flask, render_template, render_template_string, request

from IPython.display import IFrame
from threading import Thread
from time import sleep

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('template.html',query='')

@app.route('/searchsimple')
def searchsimple():
    return render_template('simplesearch.html')

@app.route('/simple_search', methods=['POST'])
def simple_search():
    query = request.form.get('query', '')
    print(query)

    output=chatbot_reponse(query,True)
    if output == 'type_1':
        return render_template('simplesearch.html')

    if 'figure' in output[1]:
        figure_output={}
        
        for i in list(output[0]):
            
            figure_output[i]=dic_figure[i]
        return render_template('simplesearch.html', result=figure_output,question=query,intent=output[1])    
            
        
    return render_template('simplesearch.html', result=output[0],question=query,intent=output[1])

@app.route('/advanced_search')
def advancedserach():
    mainConcept={'name':'','description':'','synonym':''}
    relatedConcepts=[{'name':'','description':'','synonym':''}]
    return render_template('advanced_search.html',mainConcept=mainConcept,relatedConcepts=relatedConcepts)

@app.route('/advanced_search', methods=['POST'])
def advanced_query():
    
    mainConcept={'name':'','description':'','synonym':''}
    relatedConcepts=[{'name':'','description':'','synonym':''}]
    query = request.form.get('query', '')
    print(query)
    output=chatbot_reponse(query,False)
    if output == 'type_2':
        render_template('advanced_search.html', mainConcept=mainConcept,relatedConcepts=relatedConcepts)
    
    mainConcept,relatedConcepts=build_concepts_from_node(output[0])
    return render_template('advanced_search.html', mainConcept=mainConcept,relatedConcepts=relatedConcepts)



def run_flask_app():
    app.run(port=5000)

# Start Flask in a separate thread
thread = Thread(target=run_flask_app)
thread.start()

# Wait a bit to make sure the server has started
sleep(2)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Nov/2023 12:41:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2023 12:41:48] "GET /searchsimple HTTP/1.1" 200 -


define identify risks?


127.0.0.1 - - [14/Nov/2023 12:41:59] "POST /simple_search HTTP/1.1" 200 -


response :
 {'text': 'define identify risks?', 'intent': {'name': 'ask_definition', 'confidence': 0.6785731315612793}, 'entities': [{'entity': 'entity', 'start': 7, 'end': 21, 'confidence_entity': 0.9114953875541687, 'value': 'identify risks', 'extractor': 'DIETClassifier', 'processors': ['EntitySynonymMapper']}], 'text_tokens': [[0, 6], [7, 15], [16, 21]], 'intent_ranking': [{'name': 'ask_definition', 'confidence': 0.6785731315612793}, {'name': 'detailed_process_explanation', 'confidence': 0.30885499715805054}, {'name': 'deny', 'confidence': 0.006309663876891136}, {'name': 'affirm', 'confidence': 0.0027364809066057205}, {'name': 'mood_unhappy', 'confidence': 0.001397429034113884}, {'name': 'ask_keyword', 'confidence': 0.0008139608544297516}, {'name': 'goodbye', 'confidence': 0.0005131621146574616}, {'name': 'mood_great', 'confidence': 0.00029921179520897567}, {'name': 'greet', 'confidence': 0.0002867276780307293}, {'name': 'bot_challenge', 'confidence': 0.00014579290291294456}], 'resp

127.0.0.1 - - [14/Nov/2023 12:42:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2023 12:42:34] "GET /advanced_search HTTP/1.1" 200 -


Explain the detailed process of  plan risk management ?


127.0.0.1 - - [14/Nov/2023 12:42:44] "POST /advanced_search HTTP/1.1" 200 -


response :
 {'text': 'explain the detailed process of  plan risk management ?', 'intent': {'name': 'detailed_process_explanation', 'confidence': 0.9991733431816101}, 'entities': [{'entity': 'entity', 'start': 33, 'end': 42, 'confidence_entity': 0.9234597682952881, 'value': 'plan risk', 'extractor': 'DIETClassifier'}], 'text_tokens': [[0, 7], [8, 11], [12, 20], [21, 28], [29, 31], [33, 37], [38, 42], [43, 53]], 'intent_ranking': [{'name': 'detailed_process_explanation', 'confidence': 0.9991733431816101}, {'name': 'deny', 'confidence': 0.00023234420223161578}, {'name': 'ask_definition', 'confidence': 0.00021658409968949854}, {'name': 'mood_unhappy', 'confidence': 0.000151984510011971}, {'name': 'affirm', 'confidence': 8.795215399004519e-05}, {'name': 'goodbye', 'confidence': 3.6905228625983e-05}, {'name': 'greet', 'confidence': 3.367474346305244e-05}, {'name': 'ask_keyword', 'confidence': 3.091777762165293e-05}, {'name': 'bot_challenge', 'confidence': 1.9717715986189432e-05}, {'name': 'm

127.0.0.1 - - [14/Nov/2023 12:43:51] "GET /advanced_search HTTP/1.1" 200 -
127.0.0.1 - - [14/Nov/2023 12:43:53] "GET / HTTP/1.1" 200 -
